# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [21]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

navegador = webdriver.Chrome()

#Passo 1: pegar a cotação do dolar

#entrar no google
navegador.get('https://www.google.com/')

navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dolar')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)

#Passo 2: pegar a cotação do euro
navegador.get('https://www.google.com/')

navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)

#Passo 3: pegar a cotação do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')

cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')
print(cotacao_ouro)

navegador.quit()

5.1605
5.559205390000001
311.97


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [23]:
#Passo 4: importar a base de dados e atualizar a base
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [29]:
#Passo 5: recalcular os preços

#atualizar cotação
#cotação do dolar
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
#cotação do euro
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
#cotação do ouro
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

# preço de compra = cotação * preço original
tabela['Preço de Compra'] = tabela['Cotação'] * tabela['Preço Original']

# preço de venda = preço de compra * margem
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.160500,5160.448395,1.40,7224.627753
1,Carro Renault,4500.00,Euro,5.559205,25016.424255,2.00,50032.848510
2,Notebook Dell,899.99,Dólar,5.160500,4644.398395,1.70,7895.477271
3,IPhone,799.00,Dólar,5.160500,4123.239500,1.70,7009.507150
4,Carro Fiat,3000.00,Euro,5.559205,16677.616170,1.90,31687.470723
5,Celular Xiaomi,480.48,Dólar,5.160500,2479.517040,2.00,4959.034080
6,Joia 20g,20.00,Ouro,311.970000,6239.400000,1.15,7175.310000


### Agora vamos exportar a nova base de preços atualizada

In [30]:
#Passo 6: exportar

tabela.to_excel('Produtos Novo.xlsx', index=False)

In [ ]:
!pip install selenium